<i> Note: This notebook is inspired by the Topic-Modeling-Latent-Dirichlet-Allocation series at: https://github.com/rhasanbd/Topic-Modeling-Latent-Dirichlet-Allocation </i>

## Latent Dirichlet Allocation - Implementation on Yelp dataset

In this notebook, we implement Latent Dirichlet Allocation(LDA) on the Yelp reviews data to carry out Topic Modelling. We use the Gensim topic modelling API https://radimrehurek.com/gensim/models/ldamodel.html. Scikit-Learn implementation is also available (we use Gensim since it provides more functionality and application like Topic Coherence Pipeline or Dynamic Topic Modeling.)

We build an **end-to-end Natural Language Processing (NLP) pipeline**, starting with raw data and running through preparing, modeling, visualization.
The steps that we will carry out involves the following:
1. Exploratory Data Analysis
2. Data Cleaning and Pre-processing
3. Topic modeling with LDA
4. Determine optimal number of Topics
5. Visualize topic model using pyLDAvis

### Yelp Review Dataset
The Yelp Review Dataset is a CSV file that contains a sub-sample of 10,000 reviews extracted from the Yelp dataset available at: https://www.yelp.com/dataset.

The review dataset contains the following fields:
- business_id : Unique identifier of business
- date : Data of review posted YYYY-MM-DD
- review_id : Unique identifier of review
- stars : Star rating (upto 4 stars)
- text : Review text
- user_id : Unique identifier of user who posted the review
- cool : Number of cool votes received
- useful : Number of useful votes received
- funny : Number of funny votes received

In [17]:
import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.DEBUG)

%pylab inline
import pandas as pd
import pickle as pk
from scipy import sparse as sp

import nltk
nltk.download('wordnet')

Populating the interactive namespace from numpy and matplotlib


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rojin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

## Load & Explore the Data

In [4]:
df = pd.read_csv('Data/yelp_academic_dataset_review_10000.csv')

# df1 = pd.read_csv('/Users/hasan/datasets/NIPS2015_Papers.csv')

df.head()

,business_id,date,review_id,stars,text,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,1/26/2011,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,7/27/2011,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
2,6oRAC4uyJCsJl1X0WZpVSA,6/14/2012,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,0hT2KtfLiobPvh6cDC8JQg,0,1,0
3,_1QQZuf4zZOyFCvXc0o6Vg,5/27/2010,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",uZetl9T0NcROGOyFfughhg,1,2,0
4,6ozycU1RpktNG2-1BroVtw,1/5/2012,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0


## Description of the Data

DataFrame’s info() method is useful to get a quick description of the data, in particular the total number of rows, and each attribute’s type and number of non-null values.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
business_id    10000 non-null object
date           10000 non-null object
review_id      10000 non-null object
stars          10000 non-null int64
text           10000 non-null object
user_id        10000 non-null object
cool           10000 non-null int64
useful         10000 non-null int64
funny          10000 non-null int64
dtypes: int64(4), object(5)
memory usage: 703.2+ KB


## Dimension the Data

Get the dimension (number of rows and columns) of the data using DataFrame's shape method.

In [6]:
print("Dimension of the data: ", df.shape)

no_of_rows = df.shape[0]
no_of_columns = df.shape[1]

print("No. of Rows: %d" % no_of_rows)
print("No. of Columns: %d" % no_of_columns)

Dimension of the data:  (10000, 9)
No. of Rows: 10000
No. of Columns: 9


## Convert the DataFrame Object into a 2D Array of Documents

We convert the documents from DataFrame object to an array of documents.

It's a 2D array in which each row reprents a document.

In [126]:
docs_array = array(df['text'])

print("Dimension of the documents array: ", docs_array.shape)

# Display the first document
#print(docs_array[0])

Dimension of the documents array:  (10000,)


## Pre-process the Data


We pre-process the data as follows. 

- Convert to lowercase 
- Tokenize (split the documents into tokens or words)
- Remove numbers, but not words that contain numbers
- Remove words that are only one character
- Lemmatize the tokens/words


### Tokenization

We tokenize the text using a regular expression tokenizer from NLTK. We remove numeric tokens and tokens that are only a single character, as they don’t tend to be useful, and the dataset contains a lot of them.


The NLTK Regular-Expression Tokenizer class "RegexpTokenizer" splits a string into substrings using a regular expression. We use the regular expression "\w+" to matche token of words. 

See the following two links for a list of regular expressions and NLTK tokenize module.
https://github.com/tartley/python-regex-cheatsheet/blob/master/cheatsheet.rst
https://www.nltk.org/api/nltk.tokenize.html


## Function to Convert the 2D Document Array into a 2D Array of Tokenized Documents

In [127]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

def docs_preprocessor(docs):
    tokenizer = RegexpTokenizer(r'\w+') # Tokenize the words.
    
    for idx in range(len(docs)):
#         docs[idx] = docs[idx].lower()  # Convert to lowercase.
        docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

    # Remove numbers, but not words that contain numbers.
    docs = [[token for token in doc if not token.isdigit()] for doc in docs]
    
    # Remove words that are only one character.
    docs = [[token for token in doc if len(token) > 3] for doc in docs]
    
    # Remove words that are only two character.
    docs = [[token for token in doc if len(token) > 4] for doc in docs]
    
    # Lemmatize all words in documents.
    lemmatizer = WordNetLemmatizer()
    docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]
  
    return docs

## Convert the 2D Document Array into a 1D Array of Tokenized Words

In [128]:
# Convert the 2D Document Array into a 1D Array of Tokenized Words
%time docs = docs_preprocessor(docs_array)

Wall time: 2.3 s


In [129]:
print("Length of the 2D Array of Tokenized Documents: ", len(docs))

#Display the first two document
print(docs[0:2])

Length of the 2D Array of Tokenized Documents:  10000
[['birthday', 'breakfast', 'excellent', 'weather', 'perfect', 'which', 'sitting', 'outside', 'overlooking', 'their', 'ground', 'absolute', 'pleasure', 'waitress', 'excellent', 'arrived', 'quickly', 'Saturday', 'morning', 'looked', 'place', 'fill', 'pretty', 'quickly', 'earlier', 'better', 'yourself', 'favor', 'their', 'Bloody', 'phenomenal', 'simply', 'pretty', 'ingredient', 'their', 'garden', 'blend', 'fresh', 'order', 'amazing', 'While', 'EVERYTHING', 'look', 'excellent', 'white', 'truffle', 'scrambled', 'vegetable', 'skillet', 'tasty', 'delicious', 'piece', 'their', 'griddled', 'bread', 'amazing', 'absolutely', 'complete', 'toast', 'Anyway'], ['people', 'review', 'about', 'place', 'please', 'everyone', 'probably', 'griping', 'about', 'something', 'their', 'fault', 'there', 'people', 'friend', 'arrived', 'about', 'Sunday', 'pretty', 'crowded', 'thought', 'Sunday', 'evening', 'thought', 'would', 'forever', 'seated', 'come', 'seatin

## Compute Bigrams/Trigrams:


When topics are very similar, we may **use phrases** rather than single/individual words to distinguis each topic. 

Thus, we compute both bigrams and trigrams. Depending on the dataset it may not be necessary to create trigrams.

Note that we only keep the **frequent** phrases (bigrams/trigrams).

#### Bigrams
Bigrams are sets of two adjacent words. Using bigrams we can get phrases like “machine_learning” in our output (spaces are replaced with underscores). Without bigrams we would only get “machine” and “learning”.

Note that in the code below, we find bigrams and then add them to the original data, because we would like to keep the words “machine” and “learning” as well as the bigram “machine_learning”.

In [130]:
from gensim.models import Phrases

# Add bigrams and trigrams to docs (only ones that appear 10 times or more).
bigram = Phrases(docs, min_count=10)
trigram = Phrases(bigram[docs])

for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)
    for token in trigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

## Remove Rare and Common Tokens/Words

We remove rare words and common words based on their document frequency. 

For example, we may remove words that appear in less than 10 documents or in more than 20% of the documents. 

In [138]:
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)
print('Number of unique words in initital documents:', len(dictionary))

# Filter out words that occur less than 10 documents, or more than 20% of the documents.
dictionary.filter_extremes(no_below=10, no_above=0.15)
print('Number of unique words after removing rare and common words:', len(dictionary))

Number of unique words in initital documents: 29286
Number of unique words after removing rare and common words: 5087


## Bag-of-Words Representation of Data


Finally, we transform the documents to a **vectorized form**. 

We simply compute the frequency of each word, including the bigrams.

In [139]:
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]

print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 5087
Number of documents: 10000


## Training the LDA Model

We use the gensim.models.LdaModel class for performing LDA.

We need to set the parameters of the LdaModel object carefully. The full list of the parameters are given:

https://radimrehurek.com/gensim/models/ldamodel.html


#### Below we discuss the setting of some of the key parameters.

- num_topics (int, optional) – The number of requested latent topics to be extracted from the training corpus.

 
LDA is an unsupervised technique, meaning that we don't know prior to running the model how many topics exits in our corpus. It depends on the data and the application. We may use the following two technique to determine the number of topics.


        Technique 1: Topic Coherence 
The main technique to determine the number of topics is **Topic coherence**:
http://svn.aksw.org/papers/2015/WSDM_Topic_Evaluation/public.pdf


        Technique 2: Visualizing Inter-Topic Distance 
Use the LDA visualization tool pyLDAvis to observe Intertopic Distance Map (discussed later). By varying the number of topics we could determine the optimal value from the visualization.

We **use both techniques** to determine the optimal number of topics.


- chunksize (int, optional) – Number of documents to be used in each training chunk.

It controls how many documents are processed at a time in the training algorithm. Increasing chunksize will speed up training, at least as long as the chunk of documents easily fit into memory. 

We set chunksize = 2000, which is more than the amount of documents. Thus, it processes all the data in one go. 

Chunksize can however influence the quality of the model.


- passes (int, optional) – Number of passes through the corpus during training.

It controls how often we train the model on the entire corpus. Another word for passes might be “epochs”. 


- iterations (int, optional) – Maximum number of iterations through the corpus when inferring the topic distribution of a corpus.

It is somewhat technical, but essentially it controls how often we repeat a particular loop over each document. 

        It is important to set the number of “passes” and “iterations” high enough.



#### How to Set "passes" and "iterations":

First, enable logging and set eval_every = 1 (however, it might slow down, so, we use None) in LdaModel. 

When training the model look for a line in the log that looks something like this:

        2020-02-25 19:07:04,716 : DEBUG : 49/403 documents converged within 400 iterations

If we set passes = 20, we will see this line 20 times. 

### Important: We need to make sure that by the final passes, most of the documents have converged. 

For example, if passes = 20 and iterations = 400, then, we should see something like following:


        2020-02-25 19:07:18,041 : INFO : PROGRESS: pass 19, at document #403/403
        2020-02-25 19:07:18,042 : DEBUG : performing inference on a chunk of 403 documents
        2020-02-25 19:07:18,627 : DEBUG : 402/403 documents converged within 400 iterations

Thus, want to choose both passes and iterations to be high enough for this to happen.


- eval_every (int, optional) – Log perplexity is estimated every that many updates. Setting this to 1 slows down training by ~2x.


- alpha ({numpy.ndarray, str}, optional): Can be set to an 1D array of length equal to the number of expected topics that expresses our a-priori belief for the each topics’ probability. 

Alternatively default prior selecting strategies can be employed by supplying a string:

        ’asymmetric’: Uses a fixed normalized asymmetric prior of 1.0 / topicno.

        ’auto’: Learns an asymmetric prior from the corpus (not available if distributed==True).
        
        
- eta ({float, np.array, str}, optional) – A-priori belief on word probability.

It can be:

        scalar for a symmetric prior over topic/word probability,

        vector of length num_words to denote an asymmetric user defined probability for each word,

        matrix of shape (num_topics, num_words) to assign a probability for each word-topic combination,

        the string ‘auto’ to learn the asymmetric prior from the data.


We set alpha = 'auto' and eta = 'auto'. Again this is somewhat technical, but essentially we are automatically learning two parameters in the model that we usually would have to specify explicitly.

In [145]:
from gensim.models import LdaModel

# Set training parameters.
num_topics = 5
chunksize = 500 # Size of the doc looked at every pass
passes = 20 # Number of passes through documents
iterations = 400 # Maximum number of iterations through the corpus when inferring the topic distribution of a corpus.
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make an index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

Wall time: 48.9 s


## Technique 1 for Determining Optimal Number of Topics: Topic Coherence

Topic Coherence is a measure used to evaluate topic models. Each such generated topic consists of words, and the topic coherence is applied to the top N words from the topic. 

Topic Coherence measures score a single topic by **measuring the degree of semantic similarity between high scoring words in the topic**. These measurements help distinguish between topics that are semantically interpretable topics and topics that are artifacts of statistical inference. 

A set of statements or facts is said to be coherent, if they support each other. Thus, a coherent fact set can be interpreted in a context that covers all or most of the facts. An example of a coherent fact set is “the game is a team sport”, “the game is played with a ball”, “the game demands great physical efforts”

Topic Coherence is defined as the average of the pairwise word-similarity scores of the words in the topic.

A good model will generate coherent topics, i.e., topics with high topic coherence scores. Good topics are topics that can be described by a short label, therefore this is what the topic coherence measure should capture.


Below we display 
- the average topic coherence and
- print the topics in order of topic coherence


We use LdaModel's "top_topics" method to get the topics with the highest coherence score the coherence for each topic.

Note that we use the “Umass” topic coherence measure here (see gensim.models.ldamodel.LdaModel.top_topics()).

In [146]:
top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

Average topic coherence: -3.1269.
[([(0.011593476, 'drink'),
   (0.011368852, 'night'),
   (0.009687498, 'table'),
   (0.00916343, 'could'),
   (0.0091171805, 'people'),
   (0.009064639, 'pretty'),
   (0.009057821, 'thing'),
   (0.008961214, 'think'),
   (0.008953973, 'friend'),
   (0.008762954, 'after'),
   (0.0077471994, 'before'),
   (0.0074814204, 'first'),
   (0.007239155, 'right'),
   (0.006789095, 'going'),
   (0.0067751314, 'again'),
   (0.0067248307, 'while'),
   (0.0066727963, 'around'),
   (0.0063811378, 'order'),
   (0.006224875, 'happy'),
   (0.006106414, 'review')],
  -1.7609037739449036),
 ([(0.016031815, 'ordered'),
   (0.014687053, 'chicken'),
   (0.013844945, 'sauce'),
   (0.013504017, 'salad'),
   (0.012765066, 'pizza'),
   (0.012104686, 'lunch'),
   (0.011776938, 'fresh'),
   (0.011679871, 'cheese'),
   (0.011498498, 'sandwich'),
   (0.009784301, 'delicious'),
   (0.009240105, 'taste'),
   (0.00912928, 'flavor'),
   (0.008288048, 'tasty'),
   (0.007930164, 'dinner')

## Technique 2 for Determining Optimal Number of Topics: Visualization

We use **pyLDAvis** to interpret the topics in a topic model that has been fit to a corpus of text data. 

It extracts information from a fitted LDA topic model to inform an interactive web-based visualization.

In [147]:
# import pyLDAvis.gensim
# pyLDAvis.enable_notebook()

# import warnings
# warnings.filterwarnings("ignore", category=DeprecationWarning) 

# pyLDAvis.gensim.prepare(model, corpus, dictionary)


## Interpretation of the Visualization 



- Left Panel: 
The labeld Intertopic Distance Map, circles represent different topics and the distance between them. Similar topics appear closer and the dissimilar topics farther. The relative size of a topic's circle in the plot corresponds to the relative frequency of the topic in the corpus. An individual topic may be selected for closer scrutiny by clicking on its circle, or entering its number in the "selected topic" box in the upper-left.



- Right Panel:
It includes the bar chart of the top 30 terms. When no topic is selected in the plot on the left, the bar chart shows the top-30 most "salient" terms in the corpus. A term's saliency is a measure of both how frequent the term is in the corpus and how "distinctive" it is in distinguishing between different topics. Selecting each topic on the right, modifies the bar chart to show the "relevant" terms for the selected topic. 

Relevence is defined as in footer 2 and can be tuned by parameter $\lambda$.
- Smaller $\lambda$ gives higher weight to the term's distinctiveness.
- larger $\lambda$ corresponds to probablity of the term occurance per topics.

Therefore, to get a better sense of terms per topic we use $\lambda = 0$.

## Display the Top Words in the Topics

In [148]:
def get_lda_topics(model, num_topics, top_words):
    word_dict = {};
    for i in range(num_topics):
        words = model.show_topic(i, topn = top_words);
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words];
    return pd.DataFrame(word_dict)

In [149]:
get_lda_topics(model, num_topics, 20)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05
0,always,drink,breakfast,ordered,store
1,staff,night,coffee,chicken,customer_service
2,friendly,table,chip,sauce,customer
3,price,could,taco,salad,going
4,burger,people,Mexican,pizza,business
5,location,pretty,cream,lunch,There
6,selection,thing,chocolate,fresh,owner
7,Great,think,salsa,cheese,through
8,Phoenix,friend,chip_salsa,sandwich,money
9,visit,after,burrito,delicious,these


## Generate Labels for the Topics

We can manually generate human-interpretable labels for each topic by looking at the terms that appear more in each topic.


We use LdaModel's "show_topic" method that returns **Word-probability pairs** for the most relevant words generated by the topic.

In [150]:
def explore_topic(lda_model, topic_number, topn, output=True):
    """
    accept a ldamodel, a topic number and topn vocabs of interest
    prints a formatted list of the topn terms
    """
    terms = []
    for term, frequency in lda_model.show_topic(topic_number, topn=topn):
        terms += [term]
        if output:
            print(u'{:30} {:.3f}'.format(term, round(frequency, 3)))
    
    return terms

In [151]:
topic_summaries = []

print(u'{:25} {}'.format(u'term', u'frequency') + u'\n')

for i in range(num_topics):
    print('\nTopic '+str(i)+' |---------------------------\n')
    tmp = explore_topic(model,topic_number=i, topn=10, output=True )
#     print tmp[:5]
    topic_summaries += [tmp[:5]]
    print

term                      frequency


Topic 0 |---------------------------

always                         0.026
staff                          0.020
friendly                       0.019
price                          0.017
burger                         0.013
location                       0.012
selection                      0.011
Great                          0.011
Phoenix                        0.010
visit                          0.010

Topic 1 |---------------------------

drink                          0.012
night                          0.011
table                          0.010
could                          0.009
people                         0.009
pretty                         0.009
thing                          0.009
think                          0.009
friend                         0.009
after                          0.009

Topic 2 |---------------------------

breakfast                      0.022
coffee                         0.022
chip                           0

## Manually Generate Topic Labels

Based on the most probable words generated by each topic, we assign human-interpretable labels for the topics.

In [152]:
top_labels = {0: 'Statistics', 1:'Numerical Analysis', 2:'Online Learning', 3:'Deep Learning'}